### PVlib

Burlington 44.475883, -73.212074.

In [1]:
import pandas as pd

import numpy as np

# pvlib imports
import pvlib

from pvlib.pvsystem import PVSystem
from pvlib.location import Location
from pvlib.modelchain import ModelChain
from pvlib.temperature import TEMPERATURE_MODEL_PARAMETERS

temperature_model_parameters = TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_glass']

# load some module and inverter specifications
sandia_modules = pvlib.pvsystem.retrieve_sam('SandiaMod')

cec_inverters = pvlib.pvsystem.retrieve_sam('cecinverter')

sandia_module = sandia_modules['Canadian_Solar_CS5P_220M___2009_']

cec_inverter = cec_inverters['ABB__MICRO_0_25_I_OUTD_US_208__208V_']

Creating the location object

In [2]:
location = Location(latitude=44.475883, longitude=-73.212074)

system = PVSystem(surface_tilt=20, surface_azimuth=200,
                  module_parameters=sandia_module,
                  inverter_parameters=cec_inverter,
                  temperature_model_parameters=temperature_model_parameters)


mc = ModelChain(system, location)

In [3]:
print(mc)

ModelChain: 
  name: None
  orientation_strategy: None
  clearsky_model: ineichen
  transposition_model: haydavies
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: sapm
  ac_model: snlinverter
  aoi_model: sapm_aoi_loss
  spectral_model: sapm_spectral_loss
  temperature_model: sapm_temp
  losses_model: no_extra_losses


run test weather data

In [4]:
weather = pd.DataFrame([[1050, 1000, 100, 30, 5]],
                       columns=['ghi', 'dni', 'dhi', 'temp_air', 'wind_speed'],
                       index=[pd.Timestamp('20200401 0900', tz='EST')])


mc.run_model(weather);

In [5]:
#angle of incidence
mc.aoi

2020-04-01 09:00:00-05:00    53.422191
Name: aoi, dtype: float64

In [6]:
mc.cell_temperature

2020-04-01 09:00:00-05:00    48.450974
dtype: float64

In [7]:
mc.dc

,i_sc,i_mp,v_oc,v_mp,p_mp,i_x,i_xx
2020-04-01 09:00:00-05:00,3.602811,3.213373,52.847374,42.290353,135.894678,3.517577,2.360787


i_sc = short circuit current <br>
i_mp = Current at maximum power <br>
v_oc = Voltage Open Circuit <br>
v_mp = Voltage at Maximum Power <br>
p_mp = Power at Maxium (DC outupt) <br>
i_x = current at module V = 0.5 defines 4th point on IV curve<br>
i_xx = current at module V = 0.5 defines 5th point on IV cur<br>

In [8]:
#ac output
mc.ac

2020-04-01 09:00:00-05:00    130.592362
dtype: float64

Albedo is the measure of the diffuse reflection of solar radiation out of the total solar radiation and measured on a scale from 0, corresponding to a black body that absorbs all incident radiation, to 1

#### Sources of Loss

can be used to in the PV watts calculator, an option for filling the modelsysten inputs for modelchain. <br>
https://pvwatts.nrel.gov/pvwatts.php <br>
Understanding these values are a great place to expand our understanding on solar systems. <br>

Soiling (%):  <br>
Shading (%):  <br>
Snow (%):  <br>
Mismatch (%):  <br>
Wiring (%):  <br>
Connections (%):  <br>
Light-Induced Degradation (%):  <br>
Nameplate Rating (%):  <br>
Age (%):  <br>
Availability (%): <br>

## Example
Looking at the problem below we can see an attempt at calling the pvlib model chain.

In [9]:
# surface_tilt = 30
# surface_azimuth = 180
# albedo = 0.2
# golden = pvlib.location.Location(meta['latitude'], meta['longitude'], tz='US/Mountain', 
#                                      altitude=meta['altitude'], name=meta['Name'].replace('"',''))
# print(golden)

# # Trying to design a 4 kw detailed solar with inverter and modules 
# sandia_modules = pvlib.pvsystem.retrieve_sam('SandiaMod')
# cec_inv = pvlib.pvsystem.retrieve_sam('cecinverter')

# module = sandia_modules['SunPower_SPR_220__PVL____2006_']
# #module = sandia_modules.Canadian_Solar_CS5P_220M___2009_
# inv = cec_inv['SMA_America__SB4000TL_US_22__208V__208V__CEC_2018_']

# #Making a system 
# system = pvlib.pvsystem.PVSystem(module_parameters = module,
#                                  inverter_parameters = inv,
#                                  surface_tilt = surface_tilt,
#                                  surface_azimuth = surface_azimuth,
#                                  albedo = albedo,
#                                  modules_per_string = 7,
#                                  strings_per_inverter = 3)

# mc_system = pvlib.modelchain.ModelChain(system, golden)
# mc_system.run_model(times = tmy_data.index, weather = tmy_data)

#### A useful comment posted in response

You're not seeing the variation you're expecting because ModelChain is not able to find irradiance data in your tmy_data and therefore runs the calculation assuming clear sky values.

My guess is that you're using unprocessed TMY data read directly from the read_tmy3 function. If so, you'll need to rename your GHI, DNI, and DHI columns in your tmy_data. ModelChain.run_model requires columns named ghi, dni, dhi, and optionally temp_air and wind_speed.

Also consider shifting your time index by 30 minutes to account for the difference between the measured hourly intervals and the modeled instantaneous solar position.
